In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

#import data
creative = pd.read_csv('creatives_c.csv')
creative.head()


,Campaign name,Ad Set Name,Ad name,Reach,Impressions,Frequency,Amount spent (EUR),"CPM (cost per 1,000 impressions)",Mobile app installs,Adds to cart,Cost per app install,Mobile app purchases,Preview link,CTR (link click-through rate),CPI,Reporting starts,Reporting ends
0,MAI - App Installs - DE - HH - iOS - AO - V02,16.01 - 50% - 1P data - AUTO - HH,DCO - VIDEO - Various - Jonas - APP - DE - V01,315265,1132242,3.6,7703.4,6.8,2136,189.0,3.6,101.0,https://www.facebook.com/?feed_demo_ad=2385308...,0.5,3.6,2023-01-01,2023-06-19
1,MAI - Installs - DE - CGN - iOS - Launch - V28...,CGN - V28.02.2023,Video-C2-V1-Laura-369-Deliverydoor-CGN,92735,417229,4.5,3675.7,8.8,725,166.0,5.1,84.0,https://www.facebook.com/?feed_demo_ad=2385367...,0.4,5.1,2023-01-01,2023-06-19
2,MAI - Installs - DE - CGN - Android - Launch -...,CGN - V28.02.2023,Video-C2-V1-Laura-369-Deliverydoor-CGN,51344,312964,6.1,3614.5,11.5,571,300.0,6.3,48.0,https://www.facebook.com/?feed_demo_ad=2385367...,0.4,6.3,2023-01-01,2023-06-19
3,MAI - Installs - DE - HH+CGN - iOS - AO - V25....,Videos - DCO | ALL CITIES,DCO - VIDEOS,194370,475482,2.4,3569.2,7.5,233,51.0,15.3,29.0,https://www.facebook.com/?feed_demo_ad=2385495...,0.2,15.3,2023-01-01,2023-06-19
4,AEO - Purchase - DE - HH+CGN - iOS - AO - V08....,08.03 - Various - 1P data - AUTO - HH,Video-C31-V2-Laura-369-50%,159773,366599,2.3,3496.0,9.5,454,40.0,7.7,19.0,https://www.facebook.com/?feed_demo_ad=2385376...,0.4,7.7,2023-01-01,2023-06-19


In [38]:
#make a copy of the data
creative_copy = creative.copy()

#drop the columns that are not needed
creative_copy.drop(['Preview link', 'Reporting starts', 'Reporting ends'], axis=1, inplace=True)

#making the headers lowercase and replacing spaces with underscores
creative_copy.columns = creative_copy.columns.str.lower().str.replace(' ', '_')
creative_copy.head()


#renaming headers to make them more readable
creative_copy.rename(columns={'amount_spent_(eur)': 'spent', 'cpm_(cost_per_1,000_impressions)': 'cpm', 'ctr_(link_click-through_rate)': 'ctr'}, inplace=True)

#correcting the decimal point across the data
creative_copy.round(2)

#checking for missing values in the data
creative_copy.isnull().sum()

#filling missing values with 0
creative_copy.fillna(0, inplace=True)

# Convert all numeric columns to float
numeric_columns = creative_copy.select_dtypes(include=['int','int64'])
creative_copy[numeric_columns.columns] = numeric_columns.astype(float)

# float with 2 decimal points for all columns
pd.options.display.float_format = '{:.1f}'.format

creative_copy.head()

,campaign_name,ad_set_name,ad_name,reach,impressions,frequency,spent,cpm,mobile_app_installs,adds_to_cart,cost_per_app_install,mobile_app_purchases,ctr,cpi
0,MAI - App Installs - DE - HH - iOS - AO - V02,16.01 - 50% - 1P data - AUTO - HH,DCO - VIDEO - Various - Jonas - APP - DE - V01,315265.0,1132242.0,3.6,7703.4,6.8,2136.0,189.0,3.6,101.0,0.5,3.6
1,MAI - Installs - DE - CGN - iOS - Launch - V28...,CGN - V28.02.2023,Video-C2-V1-Laura-369-Deliverydoor-CGN,92735.0,417229.0,4.5,3675.7,8.8,725.0,166.0,5.1,84.0,0.4,5.1
2,MAI - Installs - DE - CGN - Android - Launch -...,CGN - V28.02.2023,Video-C2-V1-Laura-369-Deliverydoor-CGN,51344.0,312964.0,6.1,3614.5,11.5,571.0,300.0,6.3,48.0,0.4,6.3
3,MAI - Installs - DE - HH+CGN - iOS - AO - V25....,Videos - DCO | ALL CITIES,DCO - VIDEOS,194370.0,475482.0,2.4,3569.2,7.5,233.0,51.0,15.3,29.0,0.2,15.3
4,AEO - Purchase - DE - HH+CGN - iOS - AO - V08....,08.03 - Various - 1P data - AUTO - HH,Video-C31-V2-Laura-369-50%,159773.0,366599.0,2.3,3496.0,9.5,454.0,40.0,7.7,19.0,0.4,7.7


In [39]:
def classify_ads_with_kpis(df):
    """
    Classify ads as 'Good' or 'Bad' based on fixed KPI thresholds
    """
    # Create a copy of dataframe 
    df_classified = df.copy()
    
    # Define metrics with fixed KPI thresholds
    metrics = {
        'cpm': {'threshold': 9.0, 'direction': 'lower'},      # $9 or less
        'ctr': {'threshold': 0.50, 'direction': 'higher'},    # 0.50% or higher  
        'cpi': {'threshold': 3.0, 'direction': 'lower'},      # $3 or less
    }
    
    # Calculate score for each metric
    for metric, rules in metrics.items():
        if rules['direction'] == 'lower':
            df_classified[f'{metric}_score'] = (
                df[metric] <= rules['threshold']
            ).astype(int)
        else:
            df_classified[f'{metric}_score'] = (
                df[metric] >= rules['threshold']
            ).astype(int)
    
    # Calculate overall score
    score_columns = [col for col in df_classified.columns if col.endswith('_score')]
    df_classified['total_score'] = df_classified[score_columns].mean(axis=1)
    
    # Classify performance
    df_classified['performance'] = df_classified['total_score'].apply(
        lambda x: 'Good' if x >= 0.6 else 'Bad'
    )
    
    # Add visualization color
    df_classified['performance_color'] = df_classified['performance'].map({
        'Good': 'green',
        'Bad': 'red'
    })
    
    return df_classified[['performance', 'performance_color'] + score_columns]

# Example usage:
if __name__ == "__main__":
    # Define score_columns globally
    score_columns = [col for col in creative_copy.columns if col.endswith('_score')]
    
    results = classify_ads_with_kpis(creative_copy)
    
    # Combine with original data
    final_df = pd.concat([creative_copy, results], axis=1)
    
    # Print the KPI thresholds used
    print("\nKPI Thresholds used for classification:")
    print("=" * 50)
    metrics = {
        'cpm': '≤ $9.00',
        'ctr': '≥ 0.50%',
        'cpi': '≤ $3.00'
    }
    for metric, threshold in metrics.items():
        print(f"{metric}: {threshold}")
    
    # Print performance summary
    print("\nClassification Summary:")
    print("=" * 50)
    print(final_df['performance'].value_counts())
    
    # Additional performance insights
    print("\nPerformance by Metric:")
    print("=" * 50)
    for metric in metrics:
        score_col = f"{metric}_score"
        passing_rate = final_df[score_col].mean() * 100
        print(f"{metric}: {passing_rate:.1f}% meeting target")
        
    # Show failing metrics summary
    print("\nFailing Metrics Analysis:")
    print("=" * 50)
    failing_metrics = final_df[final_df['performance'] == 'Bad'][score_columns].mean() * 100
    print("Percentage passing each metric in poorly performing ads:")
    for metric, pass_rate in failing_metrics.items():
        print(f"{metric.replace('_score', '')}: {pass_rate:.1f}%")


KPI Thresholds used for classification:
cpm: ≤ $9.00
ctr: ≥ 0.50%
cpi: ≤ $3.00

Classification Summary:
performance
Bad     60
Good    14
Name: count, dtype: int64

Performance by Metric:
cpm: 67.6% meeting target
ctr: 17.6% meeting target
cpi: 13.5% meeting target

Failing Metrics Analysis:
Percentage passing each metric in poorly performing ads:


In [40]:
final_df.head()

,campaign_name,ad_set_name,ad_name,reach,impressions,frequency,spent,cpm,mobile_app_installs,adds_to_cart,cost_per_app_install,mobile_app_purchases,ctr,cpi,performance,performance_color,cpm_score,ctr_score,cpi_score
0,MAI - App Installs - DE - HH - iOS - AO - V02,16.01 - 50% - 1P data - AUTO - HH,DCO - VIDEO - Various - Jonas - APP - DE - V01,315265.0,1132242.0,3.6,7703.4,6.8,2136.0,189.0,3.6,101.0,0.5,3.6,Good,green,1,1,0
1,MAI - Installs - DE - CGN - iOS - Launch - V28...,CGN - V28.02.2023,Video-C2-V1-Laura-369-Deliverydoor-CGN,92735.0,417229.0,4.5,3675.7,8.8,725.0,166.0,5.1,84.0,0.4,5.1,Bad,red,1,0,0
2,MAI - Installs - DE - CGN - Android - Launch -...,CGN - V28.02.2023,Video-C2-V1-Laura-369-Deliverydoor-CGN,51344.0,312964.0,6.1,3614.5,11.5,571.0,300.0,6.3,48.0,0.4,6.3,Bad,red,0,0,0
3,MAI - Installs - DE - HH+CGN - iOS - AO - V25....,Videos - DCO | ALL CITIES,DCO - VIDEOS,194370.0,475482.0,2.4,3569.2,7.5,233.0,51.0,15.3,29.0,0.2,15.3,Bad,red,1,0,0
4,AEO - Purchase - DE - HH+CGN - iOS - AO - V08....,08.03 - Various - 1P data - AUTO - HH,Video-C31-V2-Laura-369-50%,159773.0,366599.0,2.3,3496.0,9.5,454.0,40.0,7.7,19.0,0.4,7.7,Bad,red,0,0,0


In [41]:
creative_new = final_df.copy()


Encoding

In [42]:
# Get all numerical columns
numerical_columns = creative_new.select_dtypes(include=['int64', 'float64']).columns

# Identify categorical columns except performance_color
categorical_columns = creative_new.select_dtypes(include=['object']).columns
categorical_columns = [col for col in categorical_columns if col != 'performance_color']

# Create dummy variables for categorical columns
encoded_columns = pd.get_dummies(creative_new[categorical_columns], dtype=int)

# Combine numerical columns, encoded categorical columns, and performance_color
creative_new_encoded = pd.concat([
    creative_new[numerical_columns],              # Numerical columns
    encoded_columns,                             # Encoded categorical columns
    creative_new['performance_color']            # Target variable
], axis=1)

# Preview the result
creative_new_encoded.head()

,reach,impressions,frequency,spent,cpm,mobile_app_installs,adds_to_cart,cost_per_app_install,mobile_app_purchases,ctr,...,ad_name_Video-C4-V1-Laura-369-ChatChitty,ad_name_Video-C4-V1-Laura-369-ChatChitty-download,ad_name_Video-C42-V2-Laura-cheap-order-CGN,ad_name_Video-C45-V3-Laura-chatchitty-order,ad_name_Video-C6-V1-Jonas-369-StoryChat,ad_name_Video-C6-V1-Jonas-369-StoryChat-download,ad_name_Video-C6-V1-Laura-369-Chatmanner,performance_Bad,performance_Good,performance_color
0,315265.0,1132242.0,3.6,7703.4,6.8,2136.0,189.0,3.6,101.0,0.5,...,0,0,0,0,0,0,0,0,1,green
1,92735.0,417229.0,4.5,3675.7,8.8,725.0,166.0,5.1,84.0,0.4,...,0,0,0,0,0,0,0,1,0,red
2,51344.0,312964.0,6.1,3614.5,11.5,571.0,300.0,6.3,48.0,0.4,...,0,0,0,0,0,0,0,1,0,red
3,194370.0,475482.0,2.4,3569.2,7.5,233.0,51.0,15.3,29.0,0.2,...,0,0,0,0,0,0,0,1,0,red
4,159773.0,366599.0,2.3,3496.0,9.5,454.0,40.0,7.7,19.0,0.4,...,0,0,0,0,0,0,0,1,0,red


Split data  

In [43]:
#import train_test_split and GridSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV

X = creative_new_encoded.drop('performance_color', axis=1) # Features
y = creative_new_encoded['performance_color'] # Target variable

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [44]:
y_test.shape

(23,)

Modelling

In [45]:
#import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier as RandomForestClassifier

#intialize the model
RF_model = RandomForestClassifier(random_state=42)

#fit the model
RF_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Prediction

In [46]:
preds = RF_model.predict(X_test)   # Predictions
X_test

,reach,impressions,frequency,spent,cpm,mobile_app_installs,adds_to_cart,cost_per_app_install,mobile_app_purchases,ctr,...,ad_name_Video-C35-V2-Jonas-nocook-tasty3 – Copy,ad_name_Video-C4-V1-Laura-369-ChatChitty,ad_name_Video-C4-V1-Laura-369-ChatChitty-download,ad_name_Video-C42-V2-Laura-cheap-order-CGN,ad_name_Video-C45-V3-Laura-chatchitty-order,ad_name_Video-C6-V1-Jonas-369-StoryChat,ad_name_Video-C6-V1-Jonas-369-StoryChat-download,ad_name_Video-C6-V1-Laura-369-Chatmanner,performance_Bad,performance_Good
4,159773.0,366599.0,2.3,3496.0,9.5,454.0,40.0,7.7,19.0,0.4,...,0,0,0,0,0,0,0,0,1,0
63,44871.0,67464.0,1.5,629.0,9.3,64.0,8.0,9.8,2.0,0.4,...,0,0,0,0,0,0,0,0,1,0
18,153758.0,294789.0,1.9,1703.6,5.8,362.0,32.0,4.7,17.0,0.3,...,0,0,0,0,0,0,0,0,1,0
0,315265.0,1132242.0,3.6,7703.4,6.8,2136.0,189.0,3.6,101.0,0.5,...,0,0,0,0,0,0,0,0,0,1
28,51617.0,147216.0,2.9,1368.7,9.3,101.0,78.0,13.6,10.0,0.2,...,0,0,0,0,0,0,0,0,1,0
73,45618.0,73902.0,1.6,506.8,6.9,84.0,15.0,6.0,6.0,0.4,...,0,0,0,0,0,0,0,0,1,0
10,74657.0,243516.0,3.3,2387.8,9.8,224.0,178.0,10.7,49.0,0.3,...,0,0,0,0,0,0,0,0,1,0
34,96896.0,206326.0,2.1,1142.2,5.5,466.0,38.0,2.5,1.0,0.6,...,0,0,0,0,0,0,0,0,0,1
12,186464.0,418081.0,2.2,2346.4,5.6,948.0,86.0,2.5,15.0,0.6,...,0,0,0,0,0,0,0,0,0,1
55,54266.0,88428.0,1.6,701.2,7.9,20.0,1.0,35.1,0.0,0.3,...,0,0,0,0,0,0,0,0,1,0


In [47]:
y_test

4       red
63      red
18      red
0     green
28      red
73      red
10      red
34    green
12    green
55      red
65    green
31      red
9       red
45      red
5       red
22      red
57      red
50      red
33      red
39    green
70      red
16    green
35      red
Name: performance_color, dtype: object

In [48]:
comparison_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': preds
})
print("Method 1: Simple DataFrame Comparison")
print(comparison_df.head(10))
print("\n")

Method 1: Simple DataFrame Comparison
   Actual Predicted
4     red       red
63    red       red
18    red       red
0   green     green
28    red       red
73    red       red
10    red       red
34  green     green
12  green     green
55    red       red




In [49]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
acc = accuracy_score(y_test, preds)    # Accuracy
print("Accuracy:", acc)

Accuracy: 1.0


In [50]:
# Select relevant columns
summary_columns = [
    'campaign_name', 'ad_set_name', 'ad_name',  # Campaign identifiers
    'cpm', 'ctr', 'cpi', 'cost_per_app_install',  # Metrics
    'performance', 'performance_color'  # Classification
]

# Create summary dataframe
summary_df = creative_new[summary_columns]

summary_df.head()

,campaign_name,ad_set_name,ad_name,cpm,ctr,cpi,cost_per_app_install,performance,performance_color
0,MAI - App Installs - DE - HH - iOS - AO - V02,16.01 - 50% - 1P data - AUTO - HH,DCO - VIDEO - Various - Jonas - APP - DE - V01,6.8,0.5,3.6,3.6,Good,green
1,MAI - Installs - DE - CGN - iOS - Launch - V28...,CGN - V28.02.2023,Video-C2-V1-Laura-369-Deliverydoor-CGN,8.8,0.4,5.1,5.1,Bad,red
2,MAI - Installs - DE - CGN - Android - Launch -...,CGN - V28.02.2023,Video-C2-V1-Laura-369-Deliverydoor-CGN,11.5,0.4,6.3,6.3,Bad,red
3,MAI - Installs - DE - HH+CGN - iOS - AO - V25....,Videos - DCO | ALL CITIES,DCO - VIDEOS,7.5,0.2,15.3,15.3,Bad,red
4,AEO - Purchase - DE - HH+CGN - iOS - AO - V08....,08.03 - Various - 1P data - AUTO - HH,Video-C31-V2-Laura-369-50%,9.5,0.4,7.7,7.7,Bad,red
